> Noms du binome: BENICHOU - BONNEFOY
>
> Prénoms du binome: Yaniv - Nicolas
>
> Note: exporter le compte rendu basé sur le notebook au format pdf


# Entropie et génération de mots de passe
* L'objectif de ce tp est de continuer à se familiariser avec la notion d'entropie, mais aussi de comprendre le lien qu'il existe entre cette mesure informationelle et la sécurité d'un générateur (humain ou executable) de mots de passes
* Ainsi, nous proposons d'étudier l'entropie d'un tel générateur, et ce en fonction du modèle probabiliste considéré pour le modéliser (contruit à partir d'une lettre, de deux lettres, de 4 lettres, ...). A l'aide de tirrages aléatoires, nous estimerons également le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle.
* A la fin de ce TP, nous considérerons un générateur de mots de passe spécifique qui générera un mot de passe en prennant **4 lettres consécutives dans un texte** (sans se soucier des espaces). Ces lettres peuvent faire parti d'un ou de plusieurs mots consécutifs.
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents

Nous chercherons aussi à comprendre (voir dernière question):
- les bonnes pratiques pour le défenseur, i.e. la personne cherchant à générer/construire un système de génération de mots de passe.
- les bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe.

**Il est important de commenter vos réponses, en utilisant des cellules markdown**


In [2]:
import numpy as np

from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

#### Modèle monogramme (une lettre) : le générateur génère des mots de passe à partir des occurences des monogrammes
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [3]:
import pandas as pd
monogramme = pd.read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values
print(freq_mono)

[0.1776 0.0823 0.0768 0.0761 0.073  0.0723 0.0681 0.0605 0.0589 0.0534
 0.036  0.0332 0.0324 0.0272 0.0134 0.0127 0.011  0.0106 0.008  0.0064
 0.0054 0.0021 0.0019 0.0007 0.     0.    ]


Q: Quelles sont les 5 lettres les plus représentées ?

In [4]:
monogramme.head()



,letters,frequency
0,E,0.1776
1,S,0.0823
2,A,0.0768
3,N,0.0761
4,T,0.0730


Ecrire une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, il est important de bien *gérer* le cas ou la probabilité est nulle).

In [5]:
def entropie(freq):
    # Convertir les fréquences en probabilités
    total = np.sum(freq)
    proba = freq / total if total > 0 else freq

    # Filtrer les probabilités nulles pour éviter les erreurs de calcul
    proba_non_nulles = proba[proba > 0]

    # Calcul de l'entropie
    ent = -np.sum(proba_non_nulles * np.log2(proba_non_nulles))
    return ent

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [6]:
entropie_8 = entropie(monogramme['frequency'])*8
entropie_8


31.676242429778334

Q: A l'aide de la fonction `np.random.choice()`, estimer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ? (note: ici le tirage n'est pas forcemment réaliste, car aléatoire, mais l'idée est surtout de mesurer le temps minimal nécessaire pour générer N mots de passes).

In [7]:
t = time.time()

# Paramètres pour la génération de mots de passe
nb_lettres_mdp = 8
nb_mdp = 100000

# Lettres disponibles et leurs probabilités
lettres = monogramme['letters']
probabilites = monogramme['frequency']

# Générer les mots de passe
mots_de_passe = [''.join(np.random.choice(lettres, nb_lettres_mdp, p=probabilites)) for _ in range(nb_mdp)]

t_100000 = time.time() - t
print(t_100000, 's')


1.262450933456421 s


Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
    On peut montrer que $G\geq 2^H/4+1$ où $H$ est l'entropie de la source (voir le papier Password_Entropy_and_Password_Quality.pdf )

Q: calculer le minorant de $G$ pour ce modèle

In [8]:
G_minorant = 2**entropie_8 / 4 + 1
G_minorant


857904864.6814457

> En moyenne, un minimum d'environ 858 millions d'essais successifs seraient nécessaires pour trouver un mot de passe généré par ce modèle, en utilisant une stratégie d'attaque optimale. 
>
> Ce calcul illustre la difficulté théorique de deviner un mot de passe généré par ce système, en se basant sur son entropie.

Q: combien de temps cela prendra-t-il pour trouver un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [9]:
# Calcul du temps nécessaire pour trouver un mot de passe en se basant sur le minorant de G
# et sur le temps pris pour générer 100000 mots de passe

# Temps pour générer 100000 mots de passe (en secondes)
temps_100000_mdp = t_100000

# Estimation du temps pour un seul mot de passe
temps_1_mdp = temps_100000_mdp / nb_mdp

# Temps total estimé pour trouver un mot de passe (en secondes)
temps_total_pour_trouver_mdp = G_minorant * temps_1_mdp

# Convertir le temps en minutes
temps_total_pour_trouver_mdp_minutes = temps_total_pour_trouver_mdp / 60
temps_total_pour_trouver_mdp_minutes

180.51046620564927

On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [10]:

bigramme = read_csv('bigramme.csv',keep_default_na=False)
freq_bi = (bigramme['frequency']).values
letters_bi = (bigramme['letters']).values

Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [11]:
bigramme.head()

,letters,frequency
0,AA,0.000252
1,AB,0.001760
2,AC,0.003482
3,AD,0.001960
4,AE,0.000200


Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [12]:
# Calcul de l'entropie pour chaque bigramme
entropie_bigramme = entropie(freq_bi)

# Calcul de l'entropie pour un mot de passe de 8 lettres
# Un mot de passe de 8 lettres contient 4 bigrammes
entropie_mdp_8_bigramme = entropie_bigramme * 4
entropie_mdp_8_bigramme

30.14226404646422

Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ? Quelle propriété théorique de l'entropie peut justifier ce constat  ?


> Dans le modèle de bigrammes, la probabilité de chaque lettre dépend de la lettre précédente. Cette dépendance réduit la variabilité et l'imprévisibilité par rapport aux monogrammes, où chaque lettre est choisie indépendamment. Moins de variabilité signifie moins d'entropie.
>
> L'entropie est une mesure de l'incertitude ou de l'imprévisibilité. Dans le cas des bigrammes, l'entropie mesure l'incertitude de deux lettres consécutives. Si certaines combinaisons de lettres sont beaucoup plus probables que d'autres, cela réduit l'incertitude globale par rapport à un modèle où chaque lettre est choisie indépendamment et avec une distribution de probabilité plus uniforme.
>
> D'après le cours, L'entropie d'une source d'information est maximale lorsque tous les événements sont également probables. Pour les monogrammes, si chaque lettre a une probabilité relativement égale d'être choisie, cela maximise l'entropie. En revanche, pour les bigrammes, la distribution des probabilités n'est pas aussi uniforme en raison des fréquences relatives des différentes paires de lettres, ce qui entraîne une entropie globalement inférieure.

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [29]:
t = time.time()

np.random.seed(0) # reproductibilité

# Configuration pour générer des mots de passe basés sur des bigrammes
nb_bigrammes_mdp = 4  # Pour un mot de passe de 8 lettres
nb_mdp = 100000

# Générer les mots de passe
mots_de_passe_bigrammes = [''.join(np.random.choice(letters_bi, nb_bigrammes_mdp, p=freq_bi)) for _ in range(nb_mdp)]

t_bigrammes_100000 = time.time() - t
print(round(t_bigrammes_100000,4), 's')

1.5856 s


Q: calculer le minorant de $G$ pour ce modèle

In [ ]:
H_bigramme = entropie_mdp_8_bigramme
G_minorant = (2**H_bigramme)/4 + 1
G_minorant

296254956.7015476

> En moyenne, un minimum de près de 296 millions d'essais successifs seraient nécessaires pour trouver un mot de passe généré par ce modèle de bigrammes, selon une stratégie d'attaque optimale. Cela illustre la complexité et la difficulté de deviner un mot de passe basé sur ce modèle.

Q: combien de temps cela prendra-t-il pour trouver un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [15]:
# TO DO

Q: **Modèle Uniforme:** si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce nouveau générateur ?

In [16]:
# TO DO

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [17]:
# TO DO

Q: calculer le minorant de $G$ pour ce modèle

In [18]:
# TO DO

Q: dans ce cas précis, quelle est la valeur exacte de $G$?

In [19]:
# TO DO

Q: combien de temps cela prendra-t-il pour trouver un mot de passe en utilisant le générateur codé précédemment ? (en minutes)

In [20]:
# TO DO

Q: implémenter une attaque pratique qui consiste à:
1. **pour le défenseur: (la personne qui génère le mot de passe)** tirer un mot de passe de 4 lettres consécutives à partir de ce texte de Victor Hugo (texteFrancais.txt) tiré des Misérables.  
2. **pour l'attaquant:** utiliser le modèle bigramme pour générer des mots de passe et minimiser le nombre d'essais. Pour cela on pourra :
    * dans un premier temps pré-calculer un **dictionnaire**, qui contriendra un nombre de MdP générés classés dans l'ordre du plus probable au moins probable et qui ne contient pas de doublons 
    * dans un deuxième temps appeler ce dictionnaire pour comparer chacune de ses entrées au mot de passe généré.
3. Il faudra faire ses tests plusieurs fois afin de d'obtenir un nombre moyens d'appel au dictionnaire nécessaire
4. Il sera intéressant de comparer le nombre trouvé à la valeur de G (qui est une borne inférieure)
5. Question annexe: Par un simple calcul, si le générateur utilisé n'est plus ce générateur mais un générateur qui tire chaque lettre de façon équiprobable, rappeler la valeur de G. Comparer cette valeur avec la valeur trouvée en utilisant la stratégie "des 4 lettres consécutives".

In [21]:
## Fonction générant un mot de passe
def get_passwd():
    text_hugo = open("texteFrancais.txt","r")
    str_hugo = str(text_hugo.read())

    # On remplace des lettres avec accent avec des lettres sans accent
    str_hugo = str_hugo.replace("Â", "A")
    str_hugo = str_hugo.replace("Ù", "U")
    str_hugo = str_hugo.replace("Ô", "O")
    size_txt = len(str_hugo)

    idx_rand = np.random.randint(size_txt-4)
    #print(idx_rand)

    psswd = str_hugo[idx_rand:idx_rand+4]
    return(psswd)

In [22]:
# Génération du dictionnaire
tab_passwd = []
# To do ...

In [23]:
# Calcul des fréquences d'apparition de  chaque mot du dictionnaire

# To do


In [24]:
# Tri des fréquences d'apparition

In [25]:
# Attaques sur 1000 mots de passes

nb_trial = 1000
vec_nb_trials = np.zeros(nb_trial)
tab_passwd = np.array(tab_passwd)

# To do

In [26]:
# Comparaison avec la valeur de G

# To do

## Conclusions 

- Définir des bonnes pratiques pour le défenseur, i.e. la personne cherchant à concevoir un système de génération de mots de passe ? 
- Définior des bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe ?

**To do**

## Un peu de lecture
Cet article montre comment des hackers, à partir de leaks de bases de mots de passes, peuvent rapidement arriver à trouver le votre:
https://arstechnica.com/information-technology/2013/05/how-crackers-make-minced-meat-out-of-your-passwords/
